# 1.Intel® Low Precision Optimization Tool（ilit）をインストール

In [ ]:
!pip install ilit

# 2.量子化するモデル（Resnet50 - FP32）をダウンロード

In [ ]:
!wget https://storage.googleapis.com/intel-optimized-tensorflow/models/resnet50_fp32_pretrained_model.pb

# 3.推論スクリプトのダウンロード

In [ ]:
!wget https://raw.githubusercontent.com/hiouchiy/IntelAI/master/tensorflow_quantization/infer_script.py

推論スクリプト（infer_script.py）について。

IntelAIのGithubレポジトリには、いわゆる公式ベンチマークツールがあるのですが、今回はあえてそちらを使わずに（というかより実践的な状況を想定して）独自の推論スクリプトを用意しました。公式ツールよりも実装が緩いため、若干性能が劣る点はご容赦ください。なお、バッチサイズは1で固定しています。

スクリプトパラメータの説明
- --input_graph・・・モデルファイルのパス
- --dataset_dir・・・画像データフォルダのパス
- --num_images・・・推論する画像枚数（この枚数を上記画像フォルダからランダムに選んで推論します。）
- --openvino・・・OpenVINOの推論エンジン上で推論する場合はこちらにTrueをセット下さい。当然ながらモデルファイルはIRをして下さい。

# 4.FP32モデルの性能確認

ここでは、推論スクリプトの実行確認を兼ねて、量子化前のFP32のモデルの性能を確認してみます。

In [ ]:
!python infer_script.py --input_graph resnet50_fp32_pretrained_model.pb --dataset_dir /imagenet/images --num_images 50

---
ここから量子化↓
# 5.モデルの入力Op、および、出力Opの名称を確認
モデル（pb）の入出力レイヤ名を[Netron](https://lutzroeder.github.io/netron/)を使って取得します。ちなみに、今回のResNet50に場合は、入力が"input"で、出力が"predict"です。

# 6.iLitにて量子化実行

iLitはあくまでもプログラミングライブラリです。従って、iLitが提供するAPIを使用して量子化を実行するためのアプリケーションを開発する必要があります。ただ、それも面倒な作業なので[iLitのGithub](https://github.com/intel/lp-opt-tool)にはサンプルコードとして、iLitのAPIを使用した量子化アプリが用意されています。以下は、その量子化アプリを利用した量子化の実行コマンドです。

In [ ]:
!python main.py --input-graph ./resnet50_fp32_pretrained_model.pb --output-graph ./resnet50_int8_pretrained_model.pb --image_size 224 --input input --output predict --data-location /imagenet/tfrecord/ --config resnet50_v1.yaml --batch-size 10 --resize_method crop --tune

実行中に各パラメータの意味を確認しましょう。

- --input-graph ./resnet50_fp32_pretrained_model.pb ・・・入力元のFP32のモデルファイルのパス
- --input input ・・・モデルの入力Opの名前
- --output predict ・・・モデルの出力Opの名前
- --output-graph ./resnet50_int8_pretrained_model.pb ・・・出力先のINT8のモデルファイルのパス
- --data-location /imagenet/tfrecord/ ・・・量子化用画像データのパス
- --image_size 224 ・・・画像データのサイズ
- --resize_method crop ・・・画像前処理としてリサイズする際の方法。デフォルトがCrop。
- --batch-size 10 ・・・バッチサイズ
- --config resnet50_v1.yaml ・・・量子化用の設定ファイル
- --tune ・・・iLitによるチューニングを行う

量子化用の設定ファイル（YAML）の中身を見てみましょう。

In [ ]:
!cat resnet50_v1.yaml

# 7.量子化後のTensorFlowモデルを実行して性能比較

それでは、量子化後のTensorFlowモデルを実行します。先ほどと同じ推論スクリプトを使用します。

Intel TensorFlowをご使用いただいていれば、アプリケーションコードを変更しなくても、INT8のモデルを自動検知し、適切なCPU命令セット（Intel VNNI等）を実行します。推論処理のスピードがどの程度向上したかをご確認下さい。

In [ ]:
!python infer_script.py --input_graph resnet50_int8_pretrained_model.pb --dataset_dir /imagenet/images  --num_images 50

TensorFlowでの作業は以上となります。

---
ここからOpenVINO↓
# 8.OpenVINOでFP32モデルをCPUに最適化（IRに変換）

ここからはIntel® OpenVINO™ Toolkitを用いた量子化方法をご紹介します。

といってもまずは、元のTensorFlowのモデル（FP32）をOpenVINOのIR（Intermidiate Repretation）形式に変換するところから実施しましょう。

In [ ]:
!python /opt/intel/openvino/deployment_tools/model_optimizer/mo.py --input_model=./resnet50_fp32_pretrained_model.pb --input_shape=[1,224,224,3]

念のため、IR(xml+bin)が生成されていることを確認します。

In [ ]:
!ls -la

更に、IRをOpenVINOの推論エンジン（IE）上で実行してみます。TensorFlowの時と同じ推論スクリプトを使用します。モデルはFP32のままですが、IRに変換することでモデルの内部構造がCPUに最適化され、大きく性能が向上したことが確認できるかと思います。

In [ ]:
!python infer_script.py --input_graph resnet50_fp32_pretrained_model.xml --dataset_dir /imagenet/images --num_images 50 --openvino

---
ここからOpenVINOで量子化↓
# 9.OpenVINOのPOTでIRを量子化
IRの量子化はOpenVINOのPOT（Post-Training Optimization Toolkit）を使用して行います。事前にPOTの[セットアップ](https://docs.openvinotoolkit.org/latest/_README.html#install_post_training_optimization_toolkit)を完了させて下さい。

その後、量子化のための各種設定を記述したConfigファイル（JSON）を準備（ダウンロード）します。

In [ ]:
!wget https://raw.githubusercontent.com/hiouchiy/IntelAI/master/tensorflow_quantization/resnet50_int8.json

次に今回使用するConfigファイルの中身を見てみましょう。

In [ ]:
!cat resnet50_int8.json

ここでPOTに関して2点補足説明です。

1. POTはAccuracyCheckerという既存ツールを前提としている

    AccuracyCheckerはその名の通り、モデルのAccuracyを計測するためのツールです。OpenVINOのIRに変換後のモデルはもちろん、変換前の形式（TensorFlow、PyTorch、ONNXなど）であっても実行可能です。POTはこのAccuracyCheckrを拡張した機能であるため、AccuracyCheckrへの依存関係があります。したがって、上記Configファイルの前半部分は、まさにAccuracyChecker用の設定になります。
より詳しくは[こちら](https://docs.openvinotoolkit.org/latest/_README.html)を参照ください。


2. POTには2つの量子化のアルゴリズムが用意されている

    量子化のアルゴリズムとして下記2つのいずれかを利用可能です。より詳しくは[こちら](https://docs.openvinotoolkit.org/latest/_compression_algorithms_quantization_README.html)
    - DefaultQuantization・・・このサンプルで利用。より量子化処理の実行時間を高速化を優先。より詳しくは[こちら](https://docs.openvinotoolkit.org/latest/_compression_algorithms_quantization_default_README.html)
    - AccuracyAwareQuantization・・・より量子化後のAccuracyを優先。時間がかかることがある。より詳しくは[こちら](https://docs.openvinotoolkit.org/latest/_compression_algorithms_quantization_accuracy_aware_README.html)

続いて、POTを使って量子化を実行します。

In [ ]:
!pot -c resnet50_int8.json

実行が成功すると、resultsというフォルダが作成されます。そして、量子化済みのIRがその中に格納されています。

results/se_resnet50_DefaultQuantization/日付日時のフォルダ/optimized/**.xml

ちなみに、POTコマンドではなく、[Pythonスクリプト](https://docs.openvinotoolkit.org/latest/_sample_README.html#how_to_run_the_sample)を書いて同様のことを実現可能することも可能です。より細かなカスタマイズを行いたい時などはぜひご利用ください

# 10.量子化後のIRを実行

下記コマンドの日時の部分（2020-10-07_12-55-36）を実際のものに書き換えてから実行ください。

In [ ]:
!python infer_script.py --input_graph "results/resnet50_int8_DefaultQuantization/2020-10-07_12-55-36/optimized/resnet50_int8.xml" --dataset_dir /imagenet/images --num_images 50 --openvino

おまけ。AccuracyChekerを使用したモデルの精度の確認方法。

In [ ]:
!pot -c resnet50_int8.json -e -d

# 11.それぞれの結果をグラフ化して比較

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

w = 0.4

tf_total_time = 26
tf_infer_time = 20
tf_i8_total_time = 17
tf_i8_infer_time = 11
ov_total_time = 17
ov_infer_time = 10
ov_i8_total_time = 9
ov_i8_infer_time = 3

Y1 = [tf_total_time - tf_infer_time, tf_i8_total_time - tf_i8_infer_time, ov_total_time - ov_infer_time, ov_i8_total_time - ov_i8_infer_time]
Y2 = [tf_infer_time, tf_i8_infer_time, ov_infer_time, ov_i8_infer_time]

X = np.arange(len(Y1))

plt.bar(X, Y1, color='gray', width=w, label='Pre/Post', align="center")
plt.bar(X, Y2, color='blue', width=w, bottom=Y1, label='Inference', align="center")

plt.legend(loc="best")
plt.title('Model Performance Comparison')
plt.ylabel("Spent time per one image (msec)")

plt.xticks(X, ['TensorFlow(FP32)','TensorFlow(INT8)','OpenVINO(FP32)','OpenVINO(INT8)'])

plt.show()

# ------ おまけ ------

左図のように、量子化後に量子化レイヤーが連続して並ぶような場合は、性能向上が期待できます。一方、右図のように量子化レイヤーとFP32レイヤーが交互に続くようなケースは、FP32→INT8、INT8→FP32の変換が都度発生し、ボトルネックとなりえるため性能向上が期待できません。解決策として、FP32の時点で量子化ツールが対応していないレイヤー（FusedBatchNormV3など）で、かつ、除去可能であれば除去してしまうことをお薦めします。
![量子化によって性能向上が期待できるケースとできないケース](https://github.com/hiouchiy/IntelAI/raw/master/tensorflow_quantization/quantization.png "量子化によって性能向上が期待できるケースとできないケース")